<a href="https://colab.research.google.com/github/franckepeixoto/IGTI-Analista-de-Machine-Learning/blob/main/M%C3%B3dulo_4_Metodologias_de_Aprendizado_%5BDesafio_do_m%C3%B3dulo%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import load_img,img_to_array
import IPython.display as display
import PIL as Image
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import os
import pathlib
!rm -rf views
!mkdir ./views
!nvidia-smi
keras.backend.clear_session()

In [ ]:
dataPath = keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz', fname='flower_photos',untar=True)
dataPath = pathlib.Path(dataPath)
display.clear_output()
dataPath

In [ ]:
dataCount = len(list(dataPath.glob('*/*.jpg')))
print('Total:\t',dataCount)

In [ ]:
classNames = np.array([item.name for item in dataPath.glob('*') if item.name !='LICENSE.txt'])
for name in classNames:
  count = len(list(dataPath.glob(name+'/*')))
  p = str(dataPath.absolute())+'/'+name+'/'
  print(count,':\t',name.strip(),'    (',p,')')

Data Pre-processing /  Visualization

In [ ]:
def show_flowers(show="",width=224,height=224, images_path ='/root/.keras/datasets/flower_photos/'):  
  cols = 25
  limit = 100
  index = 0
  images = list()
  vertical_images=[]
  images_path = images_path+show+'/'
  for path in os.listdir(images_path):
    index=index+1
    if index%limit==0:
        break
    image = load_img(images_path+path, target_size=(width,height))
    image= img_to_array(image) #to numpy
    image_height, image_width, image_channel = image.shape
    horizontal_side = np.ones((image_height, 5,  image_channel), dtype=np.float32)*255    
    images.append(image)
    images.append(horizontal_side)
    if index%cols==0:
      horizontal_image = np.hstack((images))
      image_height, image_width, image_channel = horizontal_image.shape
      vertical_side = np.ones((5, image_width,  image_channel), dtype=np.float32)*255
      vertical_images.append(horizontal_image)
      vertical_images.append(vertical_side)
      images=list()
  gallery=np.vstack((vertical_images)) 
  plt.figure(figsize=(20,20))
  plt.xticks([])
  plt.yticks([])
  plt.title("{} amostragem de {} \nCaminho: {}".format(limit, show.upper(),images_path))
  plt.imshow(gallery.astype(np.uint8))


In [ ]:
for name in classNames:
  show_flowers(show=name)

In [ ]:
keras.backend.clear_session()
batch_size=128
validation_split=0.2
image_width = 224
image_height = 224
dataset_size = int(dataCount-(dataCount*0.3))
validation_size =  dataset_size
train_size = dataset_size

train_path = '/root/.keras/datasets/flower_photos/'
train_data_generator = ImageDataGenerator(rescale=1./255, 
                                          rotation_range=40,
                                          width_shift_range=0.2,height_shift_range=0.2,
                                          shear_range=0.2,zoom_range=0.2,
                                          fill_mode='nearest',
                                          horizontal_flip=True,validation_split=validation_split)

train_datagenerator = train_data_generator.flow_from_directory(train_path,
                                                    target_size=(image_width,image_height ),
                                                    class_mode="categorical",

                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    subset='training')


val_datagenerator = train_data_generator.flow_from_directory(train_path,
                                                     target_size=(image_width,image_height),
                                                     class_mode="categorical",
                                                     shuffle=True,
                                                     batch_size=batch_size,
                                                     subset='validation')
print(len(classNames))

In [ ]:
!ls -l /root/.keras/datasets/flower_photos/tulips/8708856019_f3be235*     

In [ ]:
img = load_img('/root/.keras/datasets/flower_photos/tulips/8708856019_f3be2353a4_n.jpg') 
x = img_to_array(img)
x = x.reshape((1,) + x.shape) 

i = 0
for batch in train_data_generator.flow(x, batch_size=1, save_to_dir='./views', save_prefix='tulips', save_format='jpeg'):
    i += 1
    if i > 20:
      break  #

In [ ]:
images_path='./views/'
fig = plt.figure(figsize = (20,10))
index = 0    
for path in os.listdir(images_path):
  index += 1
  plt.subplot(5, 5, index)
  test_image = load_img(images_path+path, target_size=(100,150))
  plt.imshow(test_image)
plt.show()


Model

In [ ]:
hub_layer =hub.KerasLayer('https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4', trainable=False)

In [ ]:
keras.backend.clear_session()
input_shape =  (image_width,image_height,3)
num_classes = len(classNames)

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=input_shape))

model.add(hub_layer)

model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(256, activation="relu"))

model.add(keras.layers.Dense(num_classes, activation="softmax", kernel_regularizer=tf.keras.regularizers.l2(0.0001)))

model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), optimizer=keras.optimizers.RMSprop(lr=0.001, decay = 1e-3, momentum = 0.3), metrics=['accuracy'])



In [ ]:
history = model.fit(train_datagenerator,  steps_per_epoch=(train_size//batch_size),epochs= 10, validation_data=val_datagenerator,validation_steps=(validation_size//batch_size) )

In [ ]:
print("Train Accuracy:\t{:.3f}%".format(history.history['accuracy'][-1]*100))
print("Val Accuracy:\t{:.3f}%".format(history.history['val_accuracy'][-1]*100))
print("Train Loss:\t{:.3f}".format(history.history['loss'][-1]))
print("Val Loss:\t{:.3f}".format(history.history['val_loss'][-1]))

Procurei no google imagens de tulipas

**Google**: "**tulips** **imagesize**:224**x**224"

In [ ]:
!rm -rf ./predict

tulips  = [
           'https://i.pinimg.com/originals/6a/0b/60/6a0b602378370a64ec1bfdc43f270746.jpg',
           'https://i.pinimg.com/originals/28/ed/49/28ed497e344619fff183780df9385ded.jpg',
           'https://i.pinimg.com/originals/c4/12/3c/c4123ce3395acc8caa717ce573886ebf.jpg'
]
!mkdir ./predict
!mkdir ./predict/images
import requests
for i,t in  enumerate(tulips):
  r = requests.get(url)  
  with open('./predict/images/'+str(i)+'.jpg', 'wb') as f:
    f.write(r.content)
!ls -l ./predict/images/

In [ ]:
Image.open('./predict/images/1.jpg')

In [ ]:
image_generator = ImageDataGenerator(rescale=1./255)
image_predict = image_generator.flow_from_directory(directory='./predict',batch_size=batch_size,
            shuffle=False,color_mode='rgb',
            target_size=(224,224),
            class_mode=None)

In [ ]:
!ls -l ./predict

In [ ]:
predicts = model.predict_classes(image_predict)

In [ ]:
print('vamos ver os indices de cada flor')
print(train_datagenerator.class_indices)
print('as três imagens foram classificadas como:')
print(predicts)

Antigo, mas usual: 

> * [Dogs x Cats Classifier | Deep Residual](https://www.kaggle.com/kernels/fork-version/38225064)

> * [CIFAR 10 - Recognition in Images / to the point.](https://www.kaggle.com/kernels/fork-version/35830305)

> * [Digit Recognizer - Tensorflow (RFF vs CNN)](https://www.kaggle.com/kernels/fork-version/38023473)

> * [Dogs x Cats Classifier Using InceptionV3](https://www.kaggle.com/kernels/fork-version/35511267)


